In [216]:
import aoc


def parse_line(line):
    d, n, color = line.split()
    direction_map = {"U": "N", "R": "E", "D": "S", "L": "W"}
    direction = direction_map[d]
    n = int(n)
    color = color[1:-1]
    return direction, n, color


data = [parse_line(line) for line in aoc.read("day18.txt").splitlines()]

# Part 1

In [217]:
gridsize = 600

grid = [["."] * gridsize for _ in range(gridsize)]

r, c = gridsize // 2, gridsize // 2
for direction, n, _ in data:
    grid[r][c] = "#"
    dr, dc = aoc.DIRECTIONS[direction]
    for _ in range(n):
        r += dr
        c += dc
        grid[r][c] = "#"

In [218]:
assert all(c == "." for c in grid[0] + grid[len(grid) - 1])

In [ ]:
import copy

total_dug = copy.deepcopy(grid)
n_dug = 0
for r, row in enumerate(grid):
    inside = False
    for c, char in enumerate(row):
        if c == 0 or c == len(grid[0]) - 1:
            assert char == "."
            continue
        if char == "#":
            n_dug += 1
            if grid[r + 1][c] == "#":
                up = True
            if grid[r - 1][c] == "#":
                down = True
            if up and down:
                inside = not inside
        if char == ".":
            up = False
            down = False
            if inside:
                n_dug += 1
                total_dug[r][c] = "#"
print(n_dug)

# Part 2

In [ ]:
def color_to_instruction(color):
    direction_map = {"0": "E", "1": "S", "2": "W", "3": "N"}
    direction = direction_map[color[-1]]
    steps = int(color[1:-1], 16)
    return direction, steps


new_instructions = [color_to_instruction(row[2]) for row in data]

In [222]:
def find_corners(instructions):
    pos = (0, 0)
    corners = []
    temp_instructions = instructions.copy()
    temp_instructions.append(instructions[0])
    clockwise_last = True  # Must start on outside turn
    for i in range(len(temp_instructions) - 1):
        dir_, steps = temp_instructions[i]
        next_dir_ = temp_instructions[i + 1][0]

        assert next_dir_ != dir_
        combi = dir_ + next_dir_
        assert combi not in {"NS", "SN", "WE", "EW"}
        clockwise_next = combi in {"NE", "ES", "SW", "WN"}

        direction = aoc.DIRECTIONS[dir_]
        # If the last turn was outside (= clockwise), then it starts once space back
        # If the next turn was outside (= clockwise), then it finishes one space further
        n_steps = steps - 1 + clockwise_last + clockwise_next

        clockwise_last = clockwise_next
        if n_steps == 0:
            continue
        pos = tuple(pos + n_steps * direction)
        corners.append(pos)

    return corners


instructions = [
    ("E", 1),
    ("S", 1),
    ("W", 1),
    ("N", 1),
]
aoc.test(find_corners(instructions), [(0, 2), (2, 2), (2, 0), (0, 0)])


instructions = [
    ("E", 3),
    ("S", 2),
    ("W", 2),
    ("S", 2),
    ("E", 2),
    ("S", 2),
    ("W", 3),
    ("N", 6),
]
aoc.test(
    find_corners(instructions),
    [(0, 4), (3, 4), (3, 2), (4, 2), (4, 4), (7, 4), (7, 0), (0, 0)],
)

instructions = [
    ("E", 3),
    ("S", 2),
    ("W", 2),
    ("S", 3),
    ("E", 1),
    ("N", 1),
    ("E", 1),
    ("S", 2),
    ("W", 3),
    ("N", 6),
]
aoc.test(
    find_corners(instructions),
    [(0, 4), (3, 4), (3, 2), (5, 2), (4, 2), (4, 4), (7, 4), (7, 0), (0, 0)],
)


instructions = [
    ("E", 1),
    ("S", 1),
    ("E", 2),
    ("N", 1),
    ("E", 2),
    ("S", 2),
    ("W", 2),
    ("S", 2),
    ("W", 1),
    ("S", 2),
    ("E", 2),
    ("S", 1),
    ("W", 3),
    ("N", 2),
    ("W", 1),
    ("N", 5),
]
aoc.test(
    find_corners(instructions),
    [
        (0, 2),
        (1, 2),
        (1, 3),
        (0, 3),
        (0, 6),
        (3, 6),
        (3, 4),
        (5, 4),
        (5, 3),
        (6, 3),
        (6, 5),
        (8, 5),
        (8, 1),
        (6, 1),
        (6, 0),
        (0, 0),
    ],
)

In [224]:
def shoelace_formula(vertices):
    corners = vertices[::-1].copy()
    # corners.append(corners[0])
    corners.insert(0, corners[-1])
    corners.append(corners[1])
    # corners.append(corners[2])

    area = 0
    alt_area = 0
    for i in range(1, len(corners) - 1):
        last = corners[i - 1]
        now = corners[i]
        next = corners[i + 1]
        area += now[1] / 2 * (last[0] - next[0])
        alt_area += (now[1] + next[1]) / 2 * (now[0] - last[0])

    return abs(area)


# corners = [(0, 0), (0, 2), (2, 2), (2, 0)]#, (1, 1), (-1, 1), (-1, 0)]
# aoc.test(shoelace_formula(corners), 4)
# corners = [(0, 0), (0, 2), (2, 2), (2, 1), (1, 1), (1, 0)]
# aoc.test(shoelace_formula(corners), 3)
# corners = [(0, 0), (0, 2), (2, 2), (2, -2), (-2, -2), (-2, 0)]
# aoc.test(shoelace_formula(corners), 12)


corners = [(1, 6), (3, 1), (7, 2), (4, 4), (8, 5)]
aoc.test(shoelace_formula(corners), 16.5)

# corners = [(0, 0), (0, 4), (2, 4), (2, 2), (4, 2), (4, 4), (6, 4), (6, 0)]
# aoc.test(shoelace_formula(corners), 26)

corners = [(0, 0), (0, 4), (3, 4), (3, 2), (4, 2), (4, 4), (7, 4), (7, 0)]
aoc.test(shoelace_formula(corners), 26)

In [ ]:
corners = find_corners(new_instructions)
shoelace_formula(corners)